In [34]:
from glob import glob
import pandas as pd

In [35]:
files = glob("../csv/例題_データアナリティクス基礎/*")
for i, f in enumerate(files):
    print(i, f)


0 ../csv/例題_データアナリティクス基礎\例題10_自由記述データ&形態素解析データ.xlsx
1 ../csv/例題_データアナリティクス基礎\例題11_乱数で発生させた分布の基本統計量.xlsx
2 ../csv/例題_データアナリティクス基礎\例題1_自転車データ.xlsx
3 ../csv/例題_データアナリティクス基礎\例題2_顧客ID付き食品購入データ.xlsx
4 ../csv/例題_データアナリティクス基礎\例題3_テレビ視聴アンケートデータ.xlsx
5 ../csv/例題_データアナリティクス基礎\例題4_5つの地域の事業所数.xlsx
6 ../csv/例題_データアナリティクス基礎\例題5_関東地方の人口データ.xlsx
7 ../csv/例題_データアナリティクス基礎\例題7_異なる授業形態による成績データ.xlsx
8 ../csv/例題_データアナリティクス基礎\例題8_遊園地テーマパーク入場者数.xlsx
9 ../csv/例題_データアナリティクス基礎\例題9_店舗別販売額.xlsx


In [36]:
questionnaire = pd.read_excel(files[4])
questionnaire.columns = ['id', 'gender', 'sports', 'interest']
questionnaire.head()

,id,gender,sports,interest
0,1,男,みる,ない
1,2,男,みる,ない
2,3,男,みる,ない
3,4,女,みない,ある
4,5,女,みる,ある


スポーツ中継 × 商品への関心 についてのクロス集計表作成

In [50]:
questionnaire_count = questionnaire.pivot_table(index='sports', columns='interest', values='id', aggfunc='count')
questionnaire_count.sort_index(ascending=False)

interest,ある,ない
sports,,
みる,68,188
みない,131,53


スポーツ中継 × 商品への関心 の構成割合についてのクロス集計表作成

In [51]:
questionnaire_ratio = questionnaire_count.apply(lambda x: x / questionnaire_count.sum().sum())
questionnaire_ratio.sort_index(ascending=False).style.format('{:.01%}', na_rep='-')

interest,ある,ない
sports,,
みる,15.5%,42.7%
みない,29.8%,12.0%


行・列にカウントの合計を追加

In [52]:
questionnaire_sum = questionnaire_count.copy()
questionnaire_sum['Total'] = questionnaire_sum.apply(lambda x : sum(x), axis=1)
questionnaire_sum.loc['Total'] = questionnaire_sum.apply(lambda x : sum(x), axis=0)
questionnaire_sum.sort_index(ascending=False)


interest,ある,ない,Total
sports,,,
みる,68,188,256
みない,131,53,184
Total,199,241,440


行・列にカウントの割合合計を追加

In [55]:
questionnaire_sum2 = questionnaire_ratio.copy()
questionnaire_sum2['Total'] = questionnaire_sum2.apply(lambda x : sum(x), axis=1)
questionnaire_sum2.loc['Total'] = questionnaire_sum2.apply(lambda x : sum(x), axis=0)
questionnaire_sum2.sort_index(ascending=False).style.format('{:.01%}', na_rep='-')

interest,ある,ない,Total
sports,,,
みる,15.5%,42.7%,58.2%
みない,29.8%,12.0%,41.8%
Total,45.2%,54.8%,100.0%


行方向の割合合計

In [56]:
questionnaire_ratio2 = questionnaire_count.copy()
questionnaire_ratio2.loc['Total'] = questionnaire_ratio2.sum()
questionnaire_ratio2 = questionnaire_ratio2.apply(lambda x: x / sum(x), axis=1)
questionnaire_ratio2['Total'] = questionnaire_ratio2.apply(lambda x: sum(x), axis=1)
questionnaire_ratio2.sort_index(ascending=False).style.format('{:.01%}', na_rep='-')

interest,ある,ない,Total
sports,,,
みる,26.6%,73.4%,100.0%
みない,71.2%,28.8%,100.0%
Total,45.2%,54.8%,100.0%


列方向の割合合計

In [58]:
questionnaire_ratio3 = questionnaire_count.copy()
questionnaire_ratio3['Total'] = questionnaire_ratio3.apply(lambda x: sum(x), axis=1)
questionnaire_ratio3 = questionnaire_ratio3.apply(lambda x: x / sum(x), axis=0)
questionnaire_ratio3.loc['Total'] = questionnaire_ratio3.sum()
questionnaire_ratio3.sort_index(ascending=False).style.format('{:.01%}', na_rep='-')

interest,ある,ない,Total
sports,,,
みる,34.2%,78.0%,58.2%
みない,65.8%,22.0%,41.8%
Total,100.0%,100.0%,100.0%


In [60]:
questionnaire.head()

,id,gender,sports,interest
0,1,男,みる,ない
1,2,男,みる,ない
2,3,男,みる,ない
3,4,女,みない,ある
4,5,女,みる,ある


性別の「スポーツ中継」に関する構成割合及び、性別の「商品への関心」に関する構成割合

In [106]:
gender_sports = questionnaire.pivot_table(index='gender', columns='sports', values='id', aggfunc='count')
gender_sports.loc['Total'] = gender_sports.sum()
gender_sports = gender_sports.apply(lambda x: x / sum(x), axis=1)
gender_sports['Total'] = gender_sports.apply(lambda x: sum(x), axis=1)
gender_sports.style.format('{:.01%}', na_rep='-')

gender_interest = questionnaire.pivot_table(index='gender', columns='interest', values='id', aggfunc='count')
gender_interest.loc['Total'] = gender_interest.sum()
gender_interest = gender_interest.apply(lambda x: x / sum(x), axis=1)
gender_interest['Total'] = gender_interest.apply(lambda x: sum(x), axis=1)
gender_interest.style.format('{:.01%}', na_rep='-')

pd.concat([gender_sports, gender_interest], axis=1).style.format('{:.01%}', na_rep='-')

,みない,みる,Total,ある,ない,Total
gender,,,,,,
女,80.4%,19.6%,100.0%,79.4%,20.6%,100.0%
男,10.0%,90.0%,100.0%,17.0%,83.0%,100.0%
Total,41.8%,58.2%,100.0%,45.2%,54.8%,100.0%


観測度数と期待度数

In [270]:
observation_freq = questionnaire_sum.copy()
observation_freq

interest,ある,ない,Total
sports,,,
みない,131,53,184
みる,68,188,256
Total,199,241,440


In [293]:
# 1:1
expected_freq11 = pd.concat([observation_freq['Total']/2, observation_freq['Total']/2, observation_freq['Total']], axis=1)
expected_freq11

,Total,Total,Total
sports,,,
みない,92.0,92.0,184
みる,128.0,128.0,256
Total,220.0,220.0,440


In [294]:
# 2:1
expected_freq12 = pd.concat([observation_freq['Total']*2/3, observation_freq['Total']*1/3, observation_freq['Total']], axis=1)
expected_freq12

,Total,Total,Total
sports,,,
みない,122.666667,61.333333,184
みる,170.666667,85.333333,256
Total,293.333333,146.666667,440


2変数に関連性がないことを仮定したときの期待度数

In [295]:
questionnaire_sum.

interest,ある,ない,Total
sports,,,
みない,131,53,184
みる,68,188,256
Total,199,241,440
